<h1 style="border: thick double #32a1ce; text-align:center;border-radius:35px">
Projet Python pour la data science: 

agrégation de la qualité d'air
</h1>

In [3]:
import pandas as pd 
from pathlib import Path
import requests
import io

**DANS CE NOTEBOOK, IL EST QUESTION DE RESUMER LA BASE DES INDICES DE L'AIR EN DONNEES ANNUELLES PAR DEPARTEMENT.**

In [4]:


# Lien direct vers le fichier CSV
url = "https://tabular-api.data.gouv.fr/api/resources/98e8387e-a53c-442d-bad3-141aed5d8a04/data/csv/"

# Envoyer une requête GET pour récupérer les données
response = requests.get(url, timeout=30)
response.raise_for_status()  # Lever une exception si la requête échoue

# Charger les données directement dans un DataFrame
data = pd.read_csv(io.StringIO(response.text))

# Afficher un aperçu du DataFrame
data.head()


,__id,date,ninsee,no2,o3,pm10
0,1,2017-01-01,0,23,16,43
1,2,2017-01-01,75,23,8,38
2,3,2017-01-01,77,19,17,44
3,4,2017-01-01,78,16,15,32
4,5,2017-01-01,91,14,16,34


In [10]:
#dimensions de la base et nombre de valeurs uniques de ninsee
print(f" {data.shape} \n {data['ninsee'].nunique()} \n {data.info()} ") 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476494 entries, 0 to 476493
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   __id    476494 non-null  object
 1   date    476494 non-null  object
 2   ninsee  476494 non-null  object
 3   no2     476494 non-null  object
 4   o3      476494 non-null  object
 5   pm10    476494 non-null  object
dtypes: object(6)
memory usage: 21.8+ MB
 (476494, 6) 
 1310 
 None 


In [11]:
# Liste des variables numériques à convertir
var_norm = ['no2', 'o3', 'pm10']

# Convertir les colonnes en type int, en gérant les erreurs et les valeurs manquantes
for col in var_norm:
    try:
        data[col] = pd.to_numeric(data[col], errors='coerce').astype('Int64')  # Type Int64 pour gérer NaN
    except KeyError:
        print(f"Colonne {col} introuvable dans le DataFrame.")

# Afficher les informations sur le DataFrame après conversion
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476494 entries, 0 to 476493
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   __id    476494 non-null  object
 1   date    476494 non-null  object
 2   ninsee  476494 non-null  object
 3   no2     476485 non-null  Int64 
 4   o3      476485 non-null  Int64 
 5   pm10    476485 non-null  Int64 
dtypes: Int64(3), object(3)
memory usage: 23.2+ MB


In [13]:
#les valeurs des indices de l'aire sont deja specifié dans la data set 
#donc il suffit de faire un filtre 
# Départements de la région parisienne (Île-de-France)
idf_departments = ['75', '77', '78', '91', '92', '93', '94', '95']

# Filtrer les données par les codes INSEE des départements de l'Île-de-France
idf_data = data[data['ninsee'].isin(idf_departments)]

# Afficher les premières lignes des données filtrées
idf_data.head()


,__id,date,ninsee,no2,o3,pm10
1,2,2017-01-01,75,23,8,38
2,3,2017-01-01,77,19,17,44
3,4,2017-01-01,78,16,15,32
4,5,2017-01-01,91,14,16,34
5,6,2017-01-01,92,23,12,32


In [15]:
# Supprimer les colonnes 'date' et '__id'
idf_data = idf_data.drop(columns=['date', '__id'])
idf_data.head()

,ninsee,no2,o3,pm10
1,75,23,8,38
2,77,19,17,44
3,78,16,15,32
4,91,14,16,34
5,92,23,12,32


Nous allons résumer ces données journalières en données annuelles par département. 
Au regard des analyses effectuées dans la 1ere partie de l'analyse descriptive, les distributions respectives de 'pm10', 'o3' et 'no2' nous indiquent des asymétrie pour 'pm10' et 'no2'. Nous résumerons donc l'information par la moyenne ou par la médiane.

In [16]:

# Résumer les données annuelles par département pour chaque variable
department_means = idf_data.groupby(['ninsee'])[['pm10', 'o3', 'no2']].agg({
    'pm10': 'median',  # Médiane pour pm10 (distribution étalée à droite)
    'o3': 'mean',      # Moyenne pour o3 (distribution presque normale)
    'no2': 'median'    # Médiane pour no2 (distribution étalée à droite)
}).reset_index()

# Afficher les premières lignes du résumé
print(department_means)


  ninsee  pm10         o3   no2
0     75  29.0  29.652055  36.0
1     77  29.0  35.389041  30.0
2     78  26.0  35.106849  27.0
3     91  27.0  34.536986  29.0
4     92  29.0  31.043836  33.0
5     93  29.0  32.013699  36.0
6     94  29.0  31.408219  34.0
7     95  27.0  35.342466  31.0


In [17]:

# Définition du répertoire de travail
HOME_DIR = Path.cwd().parent
DATA_DIR = Path(HOME_DIR, "data")
print(f"Work directory: {HOME_DIR} \nData directory: {DATA_DIR}")

# Vérifier si le dossier "data" existe
if not DATA_DIR.exists():
    raise FileNotFoundError(f"Le dossier 'data' n'existe pas dans : {DATA_DIR}")

# Définir le chemin du fichier à enregistrer
file_path = DATA_DIR / "department_means.csv"

# Sauvegarder le DataFrame dans le fichier CSV
department_means.to_csv(file_path, index=False)

print(f"Fichier sauvegardé avec succès dans {file_path}.")


Work directory: c:\Users\HP\Documents\Cours 2AD ENSAE\Semestre 1\Python pour data science\indice de l'air\projet-Python\projet 
Data directory: c:\Users\HP\Documents\Cours 2AD ENSAE\Semestre 1\Python pour data science\indice de l'air\projet-Python\projet\data
Fichier sauvegardé avec succès dans c:\Users\HP\Documents\Cours 2AD ENSAE\Semestre 1\Python pour data science\indice de l'air\projet-Python\projet\data\department_means.csv.
